In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Model Monitoring for Model Outside Vertex AI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring_v2/model_monitoring_for_model_outside_vertex.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在 Colab 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2model_monitoring_v2%2model_monitoring_for_model_outside_vertex.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> 在 Colab Enterprise 中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring_v2/model_monitoring_for_model_outside_vertex.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在 Workbench 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring_v2/model_monitoring_for_model_outside_vertex.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在 GitHub 上查看
    </a>
  </td>
</table>

## 概览

本教程演示了如何使用Python的Vertex AI SDK为您的模型设置Vertex AI Model Monitoring V2。 Model Monitoring V2现在支持Vertex AI之外的模型（允许您在Vertex AI中注册一个引用/占位模型而无需工件）。此功能也可以扩展到特征存储监视。

### 目标

执行的步骤包括以下内容：

- 为 Vertex AI 外部的模型注册一个参考/占位符模型
- 创建一个模型监视器
- 运行按需模型监控作业
- 持续模型监控


### 成本

在公共预览期间，Vertex AI 模型监控 v2 是免费的，但您仍然会收到以下 Google Cloud 服务的账单：

* [BigQuery](https://cloud.google.com/bigquery/pricing)
* [Cloud Storage](https://cloud.google.com/storage/pricing)

开始吧

安装Vertex AI SDK和其他必需的软件包

In [ ]:
! pip3 install --upgrade --quiet \
    google-cloud-bigquery \
    pandas \
    pandas_gbq \
    pyarrow \
    tensorflow_data_validation[visualization] \
    google-cloud-aiplatform

请检查 google-cloud-aiplatform 的版本是否为 1.51.0 或更高版本。

In [ ]:
from google.cloud import aiplatform

aiplatform.__version__

重新启动运行时（仅限Colab）

为了使用新安装的包，您必须在Google Colab上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️内核即将重新启动。在继续下一步之前，请等待它完成。⚠️</b>
</div>

### 在笔记本环境中进行身份验证（仅适用于Colab）

在 Google Colab 上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置谷歌云项目信息并初始化 Vertex AI SDK

要开始使用 Vertex AI，您必须拥有一个现有的谷歌云项目并[启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


import os

import vertexai

! gcloud config set project $PROJECT_ID
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
! gcloud config set ai/region $LOCATION

vertexai.init(project=PROJECT_ID, location=LOCATION)

开始模型监控教程##

### 步骤 1: 创建一个云存储桶

创建一个云存储桶，用于存储诸如数据集等中间产物。

In [ ]:
# Create a Cloud Storage bucket
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶不存在时才能运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

### 步骤 2：在 Vertex AI 中创建一个占位模型

您可以在 Vertex AI 外部为您的模型创建一个引用/占位模型，仅使用显示名称即可。

In [ ]:
import google.cloud.aiplatform as aiplatform

MODEL_NAME = "penguins"  # @param {type:"string"}

model = aiplatform.Model.upload(display_name=MODEL_NAME, sync=True)

### 步骤3：准备您的基线和目标数据集

我们建议使用BigQuery存储您的生产数据集。请确保特征存储在单独的列中。以下是一个示例BigQuery模式：

注：如果您希望设置具有时间规范的连续监视，例如一个时间窗口，需要一个时间戳列。

为本教程创建一些虚假服务数据。

In [ ]:
import numpy as np
import pandas as pd

# Define the number of rows
num_random = 100000

data = {
    "island": np.random.randint(0, 3, size=num_random),
    "culmen_length_mm": np.random.normal(50, 3, num_random),
    "culmen_depth_mm": np.random.normal(20, 3, num_random),
    "flipper_length_mm": np.random.randint(160, 250, size=num_random),
    "body_mass_g": np.random.randint(3000, 8000, size=num_random),
    "sex": np.random.randint(0, 3, size=num_random),
    "predicted_species": np.random.randint(0, 6, size=num_random),
}

# Create a DataFrame from the generated data
df = pd.DataFrame(data)

# Define the time range (start and end dates) in UTC
# now-24h ~ now + 24h
start_date = pd.Timestamp.utcnow() - pd.Timedelta(days=1)
end_date = pd.Timestamp.utcnow() + pd.Timedelta(days=1)

# Generate a list to store the random timestamps
random_timestamps = []

# Generate random timestamps and add them to the list
for _ in range(num_random):
    random_seconds = np.random.randint((end_date - start_date).total_seconds())
    random_timestamp = start_date + pd.Timedelta(seconds=random_seconds)
    # Format the timestamp as a string with microseconds
    formatted_timestamp = random_timestamp.strftime("%Y-%m-%d %H:%M:%S.%f UTC")
    random_timestamps.append(formatted_timestamp)

df["timestamp"] = random_timestamps

df.to_csv("production.csv", index=False)

创建一个BigQuery数据集，并将假数据加载到一个表中。

In [ ]:
import pandas as pd

TIMESTAMP = pd.Timestamp.utcnow().strftime("%Y%m%d%H%M%S")

FAKE_DATA_BQ_DATASET = f"penguins_production_{TIMESTAMP}"
!bq mk --dataset $PROJECT_ID:$FAKE_DATA_BQ_DATASET

In [ ]:
FAKE_DATA_BQ_TABLE = f"{FAKE_DATA_BQ_DATASET}.data"
!bq load --autodetect --source_format=CSV $FAKE_DATA_BQ_TABLE "production.csv"

检查服务日志表。

In [ ]:
import pandas as pd

query_string = f"SELECT * FROM `{FAKE_DATA_BQ_TABLE}` ORDER BY timestamp DESC LIMIT 10"
pd.read_gbq(query_string, project_id=PROJECT_ID)

### 步骤 4：创建模型监视器

创建一个模型监视器，将监视详细信息与您刚刚创建的模型版本关联起来。

定义模型监控模式

监控模式是模型监控器的必需配置。该模式包括输入特征名称、预测输出和基准真值（如果可用），以及它们各自的数据类型。

**注意：对于自动机器学习表格（回归/分类），模式是可选的，因为当可用时会自动获取（如果模型监控无法检索到模式，则您需要提供自己的模式）。**

##### 您可以按如下方式手动定义模型模式：

In [ ]:
from vertexai.resources.preview import ml_monitoring

MODEL_MONITORING_SCHEMA = ml_monitoring.spec.ModelMonitoringSchema(
    feature_fields=[
        ml_monitoring.spec.FieldSchema(name="island", data_type="categorical"),
        ml_monitoring.spec.FieldSchema(name="culmen_length_mm", data_type="float"),
        ml_monitoring.spec.FieldSchema(name="culmen_depth_mm", data_type="float"),
        ml_monitoring.spec.FieldSchema(name="flipper_length_mm", data_type="integer"),
        ml_monitoring.spec.FieldSchema(name="body_mass_g", data_type="integer"),
        ml_monitoring.spec.FieldSchema(name="sex", data_type="categorical"),
    ],
    prediction_fields=[
        ml_monitoring.spec.FieldSchema(
            name="predicted_species", data_type="categorical"
        )
    ],
)

或者，如果你有大量的特征，你可以使用`transform_schema_from_bigquery`方法来检索模式，并根据需要进行修改：

In [ ]:
from vertexai.resources.preview.ml_monitoring.spec import schema

MODEL_MONITORING_SCHEMA = schema.transform_schema_from_bigquery(
    query=f"select * except(timestamp) from {FAKE_DATA_BQ_TABLE}",
    prediction_fields="predicted_species",
)
print(MODEL_MONITORING_SCHEMA.to_json())

修改架构：

In [ ]:
# Change feature `island` to categorical
MODEL_MONITORING_SCHEMA.feature_fields[0].data_type = "categorical"
# Change feature `sex` to categorical
MODEL_MONITORING_SCHEMA.feature_fields[5].data_type = "categorical"
# Change prediction output `predicted_species` to categorical
MODEL_MONITORING_SCHEMA.prediction_fields[0].data_type = "categorical"

In [ ]:
print(MODEL_MONITORING_SCHEMA.to_json())

（可选）定义训练数据集

训练数据集可以作为基线数据集来计算监控指标。您可以在模型监视器中注册训练数据集。

In [ ]:
from vertexai.resources.preview import ml_monitoring

# Copy files to your projects gs bucket to avoid permission issues.
# Ignore any error(s) for bucket already exists.
PUBLIC_TRAINING_DATASET = (
    "gs://cloud-samples-data/vertex-ai/model-monitoring/penguins/penguins_training.csv"
)
TRAINING_URI = f"{BUCKET_URI}/model-monitoring/penguins/penguins_training.csv"

! gsutil copy $PUBLIC_TRAINING_DATASET $TRAINING_URI

TRAINING_DATASET = ml_monitoring.spec.MonitoringInput(
    gcs_uri=TRAINING_URI, data_format="csv"
)

创建一个模型监控资源

一个模型监视器是一个顶级资源，用于管理您的度量和模型监视任务。

In [ ]:
from vertexai.resources.preview import ml_monitoring

my_model_monitor = ml_monitoring.ModelMonitor.create(
    project=PROJECT_ID,
    location=LOCATION,
    display_name="penguins_model_monitor",
    model_name=model.resource_name,
    model_version_id="1",
    training_dataset=TRAINING_DATASET,
    model_monitoring_schema=MODEL_MONITORING_SCHEMA,
)
MODEL_MONITOR_ID = my_model_monitor.name
print(f"MODEL MONITOR {MODEL_MONITOR_ID} created.")

### 步骤5：运行按需模型监控作业

#### 定义监控目标配置

对于表格模型，模型监控支持以下目标：

* **输入特征漂移检测**

    模型监控提供了类别和数值特征类型的漂移分析，支持以下指标：

    * 类别特征：`Jensen Shannon Divergence`、`L Infinity`
    * 数值特征：`Jensen Shannon Divergence`

    您可以通过在 `ml_monitoring.spec.DataDriftSpec` 规范的 `features` 字段中指定感兴趣的特征来选择分析。如果未指定，模型模式中的所有输入特征都将被分析。此外，您可以选择为类别或数值特征设置默认阈值，或者为单个特征指定阈值。如果检测到漂移超过阈值，则会通过电子邮件或其他通知渠道发送警报。

* **预测输出漂移检测**

    与输入特征漂移检测类似，预测输出漂移检测识别预测输出中的数据漂移。

* **特征归因漂移检测**

    对于在 Vertex AI 之外的模型而言，没有模型工件或容器信息，因此不支持特征归因分数漂移检测。

在下面的例子中，我们设置了`FEATURE_DRIFT_SPEC`和`PREDICTION_OUTPUT_DRIFT_SPEC`，这是在`tabular_objective_spec`中组合的。

In [ ]:
from vertexai.resources.preview import ml_monitoring

DEFAULT_THRESHOLD_VALUE = 0.001

FEATURE_THRESHOLDS = {
    "culmen_length_mm": DEFAULT_THRESHOLD_VALUE,
    "body_mass_g": 0.002,
}

FEATURE_DRIFT_SPEC = ml_monitoring.spec.DataDriftSpec(
    categorical_metric_type="l_infinity",
    numeric_metric_type="jensen_shannon_divergence",
    default_categorical_alert_threshold=0.001,
    default_numeric_alert_threshold=0.002,
    feature_alert_thresholds=FEATURE_THRESHOLDS,
)

PREDICTION_OUTPUT_DRIFT_SPEC = ml_monitoring.spec.DataDriftSpec(
    categorical_metric_type="l_infinity",
    numeric_metric_type="jensen_shannon_divergence",
    default_categorical_alert_threshold=0.001,
    default_numeric_alert_threshold=0.001,
)

定义警报通知和指标输出规范。

我们支持各种通知方法：

* 邮件
* [通知渠道](https://cloud.google.com/monitoring/support/notification-options)
* [云日志](https://cloud.google.com/logging/docs?_gl=1*tdcri2*_up*MQ..&gclid=Cj0KCQjwir2xBhC_ARIsAMTXk84diOnqqpDckjOZUas26cUXUgEAgEGT9uFpz9tTvkfUjmVnRs7lQuwaAjiwEALw_wcB&gclsrc=aw.ds)  

在这个笔记本中，我们以电子邮件为例。

In [ ]:
import os

from vertexai.resources.preview import ml_monitoring

EMAIL = "[your-email-address]"  # @param {type:"string"}
if os.getenv("IS_TESTING"):
    EMAIL = "noreply@google.com"

NOTIFICATION_SPEC = ml_monitoring.spec.NotificationSpec(
    user_emails=[EMAIL],
)

OUTPUT_SPEC = ml_monitoring.spec.OutputSpec(gcs_base_dir=BUCKET_URI)

运行模型监测作业

让我们开始对特征漂移检测的监控工作（比较训练数据和提供数据）。
在这个例子中，训练数据是来自Google Cloud Storage的CSV文件，而提供数据来自BigQuery。我们支持两种连接选项：

* table_uri：从表中提取所有特征。
* query：使用SQL查询，您可以选择您感兴趣的特征进行分析。如果您想指定数据窗口或设置连续监控，请确保包括时间戳列。

示例1: 使用BigQuery表URI

In [ ]:
import pandas as pd
from vertexai.resources.preview import ml_monitoring

TIMESTAMP = pd.Timestamp.utcnow().strftime("%Y%m%d%H%M%S")
JOB_DISPLAY_NAME = f"penguins_model_monitoring_job_{TIMESTAMP}"
model_monitoring_job_1 = my_model_monitor.run(
    display_name=JOB_DISPLAY_NAME,
    baseline_dataset=TRAINING_DATASET,
    target_dataset=ml_monitoring.spec.MonitoringInput(
        table_uri=f"bq://{PROJECT_ID}.{FAKE_DATA_BQ_TABLE}"
    ),
    tabular_objective_spec=ml_monitoring.spec.TabularObjective(
        # Input feature drift spec.
        feature_drift_spec=FEATURE_DRIFT_SPEC,
        # Prediction output drift spec.
        prediction_output_drift_spec=PREDICTION_OUTPUT_DRIFT_SPEC,
    ),
    notification_spec=NOTIFICATION_SPEC,
    output_spec=OUTPUT_SPEC,
)

例2：使用SQL查询

让我们使用SQL查询创建另一个模型监控任务。如果您想指定时间规范，请确保包括时间戳列。

In [ ]:
TIMESTAMP = pd.Timestamp.utcnow().strftime("%Y%m%d%H%M%S")
JOB_DISPLAY_NAME = f"penguins_model_monitoring_job_{TIMESTAMP}"
model_monitoring_job_2 = my_model_monitor.run(
    display_name=JOB_DISPLAY_NAME,
    baseline_dataset=TRAINING_DATASET,
    target_dataset=ml_monitoring.spec.MonitoringInput(
        query=f"select island, culmen_length_mm, body_mass_g, predicted_species, timestamp from {PROJECT_ID}.{FAKE_DATA_BQ_TABLE}",
        timestamp_field="timestamp",
        window="2h",
    ),
    tabular_objective_spec=ml_monitoring.spec.TabularObjective(
        # Input feature drift spec.
        feature_drift_spec=FEATURE_DRIFT_SPEC,
        # Prediction output drift spec.
        prediction_output_drift_spec=PREDICTION_OUTPUT_DRIFT_SPEC,
    ),
    notification_spec=NOTIFICATION_SPEC,
    output_spec=OUTPUT_SPEC,
)

In [ ]:
my_model_monitor.list_jobs()

### 步骤6：等待模型监控作业完成并验证结果。

检查邮件

一旦模型监控工作开始运行（在批量预测工作完成后开始），您将收到一封如下的电子邮件：

<img src="https://services.google.com/fh/files/misc/create_job_email.png" />

一旦监控工作完成，如果检测到任何异常，您将会收到类似以下内容的电子邮件： 

<img src="https://services.google.com/fh/files/misc/place_holder_job_anomalies.png" />

检查监控指标：Google云控制台

要查看[Google Cloud控制台](https://console.cloud.google.com/vertex-ai/model-monitoring/model-monitors)中的模型监控指标，请转到**Vertex AI**下的**监控**选项卡。

<img src="https://services.google.com/fh/files/misc/place_holder_job_details.gif" />

检查监控指标：Cloud存储桶

运行以下命令以查看存储在Cloud存储桶中的模型监控指标。

In [ ]:
try:
    my_model_monitor.show_feature_drift_stats(model_monitoring_job_1.name)
except Exception as e:
    print(e)

In [ ]:
try:
    my_model_monitor.show_output_drift_stats(model_monitoring_job_1.name)
except Exception as e:
    print(e)

In [ ]:
try:
    my_model_monitor.show_feature_drift_stats(model_monitoring_job_2.name)
except Exception as e:
    print(e)

In [ ]:
try:
    my_model_monitor.show_output_drift_stats(model_monitoring_job_2.name)
except Exception as e:
    print(e)

### 步骤7：安排连续的模型监控

要设置连续的模型监控，请按照以下示例创建日程安排。您可以为您的模型监视器创建多个日程安排。

以下示例监视输入特征和预测输出的漂移。日程安排配置为每小时在整点启动模型监控作业，比如在00:00、01:00等时刻。每个作业分析前一个小时窗口内收集的数据。例如，如果一个作业计划于上午6点执行，则它将分析从上午5点到6点收集的数据。

In [ ]:
# Every 1 hour at :00, for example 1:00, 2:00..
CRON = "0 * * * *"  # @param {type:"string"}
SCHEDULE_DISPLAY_NAME = "penguins-continous-drift-detection"

In [ ]:
model_monitoring_schedule = my_model_monitor.create_schedule(
    display_name=SCHEDULE_DISPLAY_NAME,
    cron=CRON,
    baseline_dataset=ml_monitoring.spec.MonitoringInput(
        table_uri=f"bq://{PROJECT_ID}.{FAKE_DATA_BQ_TABLE}",
        timestamp_field="timestamp",
        window="1h",
        offset="1h",
    ),
    target_dataset=ml_monitoring.spec.MonitoringInput(
        table_uri=f"bq://{PROJECT_ID}.{FAKE_DATA_BQ_TABLE}",
        timestamp_field="timestamp",
        window="1h",
    ),
    tabular_objective_spec=ml_monitoring.spec.TabularObjective(
        # Input feature drift spec.
        feature_drift_spec=FEATURE_DRIFT_SPEC,
        # Prediction output drift spec.
        prediction_output_drift_spec=PREDICTION_OUTPUT_DRIFT_SPEC,
    ),
    notification_spec=ml_monitoring.spec.NotificationSpec(
        user_emails=[EMAIL],
    ),
    output_spec=ml_monitoring.spec.OutputSpec(gcs_base_dir=BUCKET_URI),
)

SCHEDULE_RESOURCE_NAME = model_monitoring_schedule.name
print(f"Schedule {SCHEDULE_RESOURCE_NAME} created.")

暂停计划

运行以下命令暂停模型监控计划:

In [ ]:
my_model_monitor.pause_schedule(SCHEDULE_RESOURCE_NAME)

恢复时间表

运行以下命令以恢复暂停的模型监控时间表：

In [ ]:
my_model_monitor.resume_schedule(SCHEDULE_RESOURCE_NAME)

更新时间表

运行以下命令以更新模型监控时间表：

In [ ]:
# Update to run every 1 hour at :30, for example 0:30, 1:30, 2:00..
my_model_monitor.update_schedule(
    schedule_name=SCHEDULE_RESOURCE_NAME, cron="30 * * * *"
)

#### 检查Google Cloud控制台中的监控计划

要在Google Cloud控制台中检查您的模型监控计划，请转到Vertex AI下的监控选项卡。

步骤8：清理

如果您不再需要您的模型监控资源，请运行以下命令来删除它们：

In [ ]:
from google.cloud import bigquery

# When no jobs are running, delete the model monitor.
my_model_monitor.delete(force=True)

# Delete the model.
model.delete()

# Delete BQ logging table.
bqclient = bigquery.Client(project=PROJECT_ID)
# Delete the dataset (including all tables)
bqclient.delete_dataset(FAKE_DATA_BQ_DATASET, delete_contents=True, not_found_ok=True)